In [2]:
#IMPORTS 
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib as mat
from cycler import cycler
from matplotlib import colors 
from numba import njit,prange 
from tqdm.notebook import tqdm
from scipy import integrate
import os
from scipy.interpolate import interp1d 
#MATPLOTLIB PERSONAL CONFIG  
plt.rcParams["figure.figsize"] = (5,5) 
plt.rcParams["lines.linewidth"] = 1.0 
plt.rcParams["axes.linewidth"]  = 1.0
plt.rcParams["axes.grid"] = "True" 
plt.rcParams["grid.alpha"] = 0.0 
plt.rcParams["figure.dpi"] = 150
plt.rcParams['xtick.labelsize'] = 14 
plt.rcParams['ytick.labelsize'] = 14 
plt.rcParams['legend.fontsize'] = 15
plt.rcParams['axes.labelsize'] = 14 
plt.rcParams['axes.spines.right'] = True 
plt.rcParams['axes.spines.top'] = True
plt.rcParams["grid.linestyle"] = "--" 
plt.rcParams["legend.frameon"] = False 
plt.rcParams["patch.facecolor"] ="white" 
plt.rcParams.update({'errorbar.capsize': 4}) 
plt.rcParams["savefig.bbox"] = 'tight'
#MATPLOTLIB IMSHOW COLORS 
#plt.rcParams["axes.prop_cycle"]= cycler('color',["#264653", "#2a9d8f","#e9c46a","#f4a261"])
#color = np.array(["#264653", "#2a9d8f","#e9c46a","#f4a261"]) 
#cmap1 = colors.LinearSegmentedColormap.from_list("mycmap", np.flip(color))
#plt.rcParams['image.cmap'] =  cmap1

mapa_teste = colors.LinearSegmentedColormap.from_list("mycmap", np.array(["red","blue","green","orange"]))

In [3]:
@njit
def compute_moments_trapz(M, mu, sigma):
    
    m = np.zeros(M)
    x = np.linspace(-0.99999, 0.99999, 100000)
       
    for i in range (M):
        
        m[i] = np.trapz(np.cos(i*np.arccos(x))*np.exp(-0.5*(x - mu)**2/(sigma**2))/(np.sqrt(2*np.pi)*sigma), x)
    
    return m

In [22]:
%time m_trapz = compute_moments_trapz(M, mu, sigma)

NameError: name 'compute_moments_trapz' is not defined

In [23]:
#print(m_trapz)

In [4]:
@njit
def f(n, x):
    
    return np.cos(n*np.arccos(x))*np.exp(-0.5*(x - mu)**2/(sigma**2))/(np.sqrt(2*np.pi)*sigma)*np.sqrt(1 - x**2)

@njit
def N(x):
    
    return np.exp(-0.5*(x - mu)**2/(sigma**2))/(np.sqrt(2*np.pi)*sigma)*np.sqrt(1 - x**2)

@njit
def normalization(num_points):
    
    n = 0
    
    for i in range (1, num_points + 1):

        xi = np.cos((2*i - 1)/(2*num_points)*np.pi)

        n += N(xi)
        
    n *= np.pi/num_points
    
    return n

@njit
def compute_moments_quadrature(M, mu, sigma, num_points):
    
    N = normalization(num_points)
    m = np.zeros(M)
    
    for j in range (M):
        for i in range (1, num_points + 1):

            xi = np.cos((2*i - 1)/(2*num_points)*np.pi)

            m[j] += f(j, xi)
        
        m[j] *= np.pi/num_points
    
    return m/N

In [5]:
mu = 0/(2.1 + 2.5/2)
sigma = 0.001

%time m_quadr = compute_moments_quadrature(2*16384, mu, sigma, 20000)

CPU times: user 1min 12s, sys: 78.6 ms, total: 1min 12s
Wall time: 1min 12s


In [26]:
np.savetxt("gaussian_moments.dat", m_quadr)